<a href="https://colab.research.google.com/github/mark910-ai/FlagEmbedding/blob/master/EmbeddingSearchValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!git clone https://github.com/mark910-ai/FlagEmbedding.git

Cloning into 'FlagEmbedding'...
remote: Enumerating objects: 10031, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 10031 (delta 35), reused 41 (delta 24), pack-reused 9961 (from 2)
Receiving objects: 100% (10031/10031), 53.07 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (5522/5522), done.
Updating files: 100% (997/997), done.


In [2]:
!pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:0

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["In this sentence: In the case of a limited company, the secretary or a director should sign the form., what is the meaning of the word: limited company?"]
sentences_2 = ["The word limited company is a ['noun'] and its definition is: a British company whose owners have to pay only a limited amount if the company gets into debt."]
sentences_3 = ["The word liturgy is a ['noun'] and its definition is: a way of praying in a religious service using a fixed order of words, prayers etc."]

embeddings_1 = model.encode(sentences_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

In [9]:
from FlagEmbedding import BGEM3FlagModel

model_path = '/content/drive/MyDrive/DSSM/encoder_only_m3_bge-m3_sd/checkpoint-10000'

model = BGEM3FlagModel(model_path,
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["In this sentence: In the case of a limited company, the secretary or a director should sign the form., what is the meaning of the word: limited company?"]
sentences_2 = ["The word limited company is a ['noun'] and its definition is: a British company whose owners have to pay only a limited amount if the company gets into debt."]
sentences_3 = ["The word liturgy is a ['noun'] and its definition is: a way of praying in a religious service using a fixed order of words, prayers etc."]

embeddings_1 = model.encode(sentences_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[[0.8564]]


In [11]:
import json
from typing import Dict, List

def load_jsonl(file_path: str, limit: int = None) -> List[Dict]:
    """
    load jsonl file
    Args:
        file_path: jsonl file path
        limit: limit the number of lines to read, default is None to read all
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if line.strip():
                data.append(json.loads(line))
                if limit and i >= limit - 1:
                    break
    return data

In [33]:
def inference(data: List[Dict]):
    from FlagEmbedding import BGEM3FlagModel

    model_path = '/content/drive/MyDrive/DSSM/encoder_only_m3_bge-m3_sd/checkpoint-10000'
    # model_path = 'BAAI/bge-m3'

    model = BGEM3FlagModel(model_path,
                        use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

    sentences_1 = [item['query'] for item in data]
    sentences_2 = [item['document'] for item in data]

    embeddings_1 = model.encode(sentences_1,
                                batch_size=12,
                                max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                                )['dense_vecs']
    embeddings_2 = model.encode(sentences_2)['dense_vecs']
    similarity = embeddings_1 @ embeddings_2.T
    # Get diagonal elements which represent similarities between corresponding pairs
    similarity_scores = similarity.diagonal()
    return similarity_scores

In [34]:
import random
data = load_jsonl('/content/FlagEmbedding/dataset/gse_train_data/data/validation_data.jsonl')
print(len(data))
# shuffle data
# random.shuffle(data)
data = data[:100]
similarity = inference(data)
threshold = 0.6
# compare similarity with label and print the accuracy
correct = 0
for i, item in enumerate(data):
    if similarity[i] > threshold and item['label'] == 1:
        correct += 1
    elif similarity[i] <= threshold and item['label'] == 0:
        correct += 1
print(f'Accuracy: {correct / len(data)}')

100928


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Accuracy: 0.63
